<div style='text-align: right; font-size: 16px;'>
<br/>
<img src="images/HWR_logo.svg" alt="drawing" width="30%"/>
<br/>
Fachbereich Duales Studium Wirtschaft • Technik<br/>
ET1031 Mathematische Grundlagen III (Signale und Systeme)<br/>
Prof. Dr. Luis Fernando Ferreira Furtado, Berlin, 10.10.2025<br/>
</div>

# Kapitel 9. Die Fourier-Transformation für kontinuierliche Zeitsignale und Systeme

<hr style="border:solid #000000 1px;height:1px;">

In [ ]:
import scipy.signal as signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import plotly.graph_objects as go
from IPython.display import Audio, display, Latex, HTML
from ipywidgets import interact, IntSlider, FloatSlider, ToggleButton
import warnings
warnings.filterwarnings("ignore")

def plot_line_spectrum(f0, Xk, sides=2, phase_type='rad', linetype='b', lwidth=1.2, color='#D50C2F'):
	"""
	------------------------------------------------------------------------------------------------------------
	This function plots the Fourier line spectrum given the coefficients

	Parameters:
		f0: a float with the base frequency
		Xk: a list of touples with the real coeficients (k, |Xk|, ∠Xk); (coeficient order, amplitude; phase)
		sides: 2-sided plot or 1-sided plot
		phase_type: a string with two unities modes 'rad' for radiants or 'deg' for degrees 
		linetype: a string with the line type per Matplotlib definitions, e.g., 'b';
		lwidth: a float number with the linewidth in points
		color: a string with the color name or hexa value
	------------------------------------------------------------------------------------------------------------
	"""  

	# Extract from Xk all frequencies, amplitudes and phase
	Xk_freqs = []
	Xk_amps = []
	Xk_phases = []
	for i in range(len(Xk)):
		Xk_freqs.append(round(Xk[i][0] * f0, 3))
		Xk_amps.append(abs(Xk[i][1]))

		# Deal with the phase for negative amplitudes
		if Xk[i][1] < 0:
			if phase_type=='rad':
				if Xk[i][2] > 0:
					Xk_phases.append(Xk[i][2] - np.pi)
				else:
					Xk_phases.append(Xk[i][2] + np.pi)
			elif phase_type=='deg':
				if Xk[i][2] > 0:
					Xk_phases.append(Xk[i][2] - 180)
				else:
					Xk_phases.append(Xk[i][2] + 180)
			else:
				print("Wrong phase_type!")
		else:
			Xk_phases.append(Xk[i][2])

	# Set the figure
	fig = plt.figure(figsize=(14, 4), facecolor='white')
	gs = GridSpec(nrows=1, ncols=2)    

	# Amplitude
	# Set the subplot for amplitude
	fig.add_subplot(gs[:, 0])	
	plt.xlabel('Frequenz [Hz]'); plt.ylabel('Amplitude')
	plt.margins(0, 0.1)
	plt.grid(which='both', axis='both', color='black', alpha=0.25, linewidth=.5)
	plt.axhline(0, color='black', alpha=0.5, linewidth=.5)
	plt.axvline(0, color='black', alpha=0.5, linewidth=.5)	
	
	# Plot the one side amplitude plot (real coeficients)
	if sides == 1:
		plt.title('Reelle Amplitudenspektren (eine Seite)')
		plt.axis([-f0/2, f0/2 + np.max(Xk_freqs), -1.2*np.max(np.abs(Xk_amps)), 1.2*np.max(np.abs(Xk_amps))])

		for k in range(len(Xk_freqs)):
			freq = Xk_freqs[k]
			amp = np.abs(Xk_amps[k])
			plt.plot([freq, freq],[0, amp], linetype, linewidth=2*lwidth, color=color)

	# Plot the two sides amplitude plot (complex coeficients)
	if sides == 2:
		plt.title('Komplexe Amplitudenspektren (zwei Seiten)')
		plt.axis([-f0/2 - np.max(Xk_freqs), f0/2 + np.max(Xk_freqs), -1.2*np.max(np.abs(Xk_amps)), 1.2*np.max(np.abs(Xk_amps))])

		for k in range(len(Xk_freqs)):
			freq = Xk_freqs[k]
			amp = np.abs(Xk_amps[k])
			if not np.isclose(freq, 0, atol=f0/2):
				amp = amp/2
			plt.plot([freq, freq],[0, amp], linetype, linewidth=2*lwidth, color=color)
			plt.plot([-freq, -freq],[0, amp], linetype, linewidth=2*lwidth, color=color)


	# Phase
	# Set the subplot for phase
	fig.add_subplot(gs[:, 1])	
	plt.xlabel('Frequenz [Hz]'); plt.ylabel('Phase (' + phase_type + ')')
	plt.margins(0, 0.1)
	plt.grid(which='both', axis='both', color='black', alpha=0.25, linewidth=.5)
	plt.axhline(0, color='black', alpha=0.5, linewidth=.5)
	plt.axvline(0, color='black', alpha=0.5, linewidth=.5)
	
	# Plot the one side phase plot (real coeficients)
	if sides == 1:
		plt.title('Reelle Phasenspektren (eine Seite)')
		plt.axis([-f0/2, 1.2*np.max(Xk_freqs), -1.2*np.max(np.abs(Xk_phases)), 1.2*np.max(np.abs(Xk_phases))])

		for k in range(len(Xk_freqs)):
			freq = Xk_freqs[k]
			phase = Xk_phases[k]
			plt.plot([freq, freq],[0, phase], linetype, linewidth=2*lwidth, color=color)

	# Plot the two sides phase plot (complex coeficients)
	if sides == 2:
		plt.title('Komplexe Phasenspektren (zwei Seiten)')
		plt.axis([-f0/2 - np.max(Xk_freqs), f0/2 + np.max(Xk_freqs), -1.2*np.max(np.abs(Xk_phases)), 1.2*np.max(np.abs(Xk_phases))]) 

		for k in range(len(Xk_freqs)):
			freq = Xk_freqs[k]
			phase = Xk_phases[k]
			if np.isclose(freq, 0, atol=f0/2):
				plt.plot([freq, freq],[0, phase], linetype, linewidth=2*lwidth, color=color)
			else:
				plt.plot([freq, freq],[0, phase], linetype, linewidth=2*lwidth, color=color)
				plt.plot([-freq, -freq],[0, -phase], linetype, linewidth=2*lwidth, color=color)

	plt.show()

def Xk_square(K, T0, tau):
	"""
	------------------------------------------------------------------------------------------------------------
	This function generates the frequencies and Fourier-coeficients for the squared function
	Parameters:
		K: an integer with the number of parameters to be generated
		T0: a float with the base period (f0 = 1/T0) 
		tau: a float with the lengh of tau (tau < T0)
		
	Return:
		Xk: a list of touples with the real coeficients (k, |Xk|, ∠Xk); (coeficient order, amplitude; phase)
	------------------------------------------------------------------------------------------------------------
	"""  

	# Initialize variables
	f0 = 1/T0
	Xk = []

	# Generate Fourier-coeficients
	for k in range(-K, K+1):
		X = (tau/T0) * np.sinc(k * f0 * tau) #* np.exp(-1j * np.pi * k * f0 * tau)
		if not np.isclose(np.abs(X), 0, atol=0.0001):
			Xk.append((k, np.abs(X), np.angle(X)))
		
	return Xk

def Xk_tri(K):
	"""
	------------------------------------------------------------------------------------------------------------
	This function generates the frequencies and Fourier-coeficients for the triangle function
	Parameters:
		K: an integer with the number of parameters to be generated
		
	Return:
		Xk: a list of touples with the real coeficients (k, |Xk|, ∠Xk); (coeficient order, amplitude; phase)
	------------------------------------------------------------------------------------------------------------
	"""  

	# Initialize variables
	Xk = []

	# Generate Fourier-coeficients
	for k in range(-K, K+1):
		if k == 0:
			X = 0.5 + 0j
			Xk.append((k, np.abs(X), np.angle(X)))
		elif (k % 2) != 0:
			X = -2/(np.pi*k)**2 + 0j
			Xk.append((k, np.abs(X), np.angle(X)))
		
	return Xk

def x_from_Xk(A, f0, Xk, t, plot_me=False):
	"""
	------------------------------------------------------------------------------------------------------------
	This function approximates the function x(t) based on the Fourier-coeficients
	Parameters:
		A: a float with the amplitude scaling
		f0: a float with the base frequency
		Xk: a list of touples with the real coeficients (k, |Xk|, ∠Xk); (coeficient order, amplitude; phase)
		t: an array with the time samples
		plot_me: a bool, if True, plot x(t) with all components in the time domain from the Fourier-coeficients
		
	Return:
		x: an approximation of x(t) for each sample t
	------------------------------------------------------------------------------------------------------------
	"""  
	
	# Initialize variables
	x = np.zeros(len(t))+0j  # Empty version of the final reconstructed function
	x_k = []                 # Components of x(t)
					
	# Reconstruct function
	for k in range(len(Xk)):
		freq = Xk[k][0] * f0
		amp = Xk[k][1]
		phase = Xk[k][2]
		complex_coefficient = A* amp * np.exp(1j * phase)
		x_k.append(complex_coefficient * np.exp(2j * np.pi * freq * t))
		x += x_k[k] 

	# Plot the reconstructed function
	if plot_me:
		fig = go.Figure()
		fig.add_trace(go.Scatter(x=t, y=x.real, mode='lines', name='x(t)', line=dict(color='#BE0000')))        
		for k in range(len(Xk)):
			if Xk[k][1] > 1E-10:
				fig.add_trace(go.Scatter(x=t, y=x_k[k].real, mode='lines', name='x(t) für n='+str(Xk[k][0]), visible='legendonly'))    
		fig.update_layout(height=400, template='plotly_white', xaxis = dict(title_text='t'))
		fig.show()    

	return x.real

def Xk_complex2real(Xk):
	"""
	------------------------------------------------------------------------------------------------------------
	This function convert complex coeficients in real coeficients
	Parameters:
		Xk: a list of touples with the complex coeficients (k, |Xk|, ∠Xk); (coeficient order, amplitude; phase)
		
	Return:
		Xk_real: a list of touples with the real coeficients (k, |Xk|, ∠Xk); (coeficient order, amplitude; phase)
	------------------------------------------------------------------------------------------------------------
	"""  
		
	# Initialize variables
	Xk_real = []

	for k in range(len(Xk)):
		if Xk[k][0] == 0:
			Xk_real.append(Xk[k])
		elif Xk[k][0] > 0:
			Xk_real.append((Xk[k][0], 2*Xk[k][1], Xk[k][2]))
		
	return Xk_real

def rect(t, tau):
    """
    ------------------------------------------------------------------------------------------------------------
    [Continuous Time]
    This function calculates an approximation of the rectangular function x(t) = ∏(t/tau)

    Parameters:
        t: a np.ndarray that represents the continuous time axis
        tau: a float with the width of the rectangular pulse

    Return:
        x: a np.ndarray with the rectangular signal x(t) = ∏(t/tau)
    ------------------------------------------------------------------------------------------------------------
    """          

    x = np.zeros(len(t))
    for k, tk in enumerate(t):
        if np.abs(tk) > tau/2.:
            x[k] = 0
        else:
            x[k] = 1

    return x

<hr style="border:solid #000000 1px;height:1px;">

#### Fourier-Transformation
__Fourier-Koeffizienten für infinitesimale Differenzfrequenzen___
<br/>

In [ ]:
def update(T0_per_tau):

	# Set parameters
	A = 1
	tau = 2
	T0 = tau * T0_per_tau
	f0 = 1/T0
	K = int(1 + 2 * T0)
	
	# Automatic calculation of resolution to make it less computational intensive for high values of T0
	resolution = T0 / 500
	t = np.arange(-(T0-tau), (T0-tau)+resolution, resolution)   

	# Calculate coeficients
	Xk = Xk_square(K, T0, tau)

	# Reconstruct the function x(t) based on the parameters
	x = x_from_Xk(A, f0, Xk, t, plot_me=False)
	x_ideal = rect(t, tau)   

	# Convert the complex coefficients to real coefficients to be then plotted
	Xk = Xk_complex2real(Xk)

	# Plot the reconstructed function in the time domain
	fig = go.Figure()
	fig.add_trace(go.Scatter(x=t, y=x, mode='lines', name='x(t) for K='+str(K)))
	fig.add_trace(go.Scatter(x=t, y=x_ideal, mode='lines', name='x(t) ideal'))
	fig.update_layout(height=400, template='plotly_white', xaxis = dict(title_text='t'))
	fig.show()    

	# Plot line spectrum
	plot_line_spectrum(f0, Xk, sides=2, lwidth=0.5)

T0_per_tau = IntSlider(min=2, max=100, value=2, description='T0/tau')
interact(update, T0_per_tau=T0_per_tau);